In [72]:
import sympy as sp

z = sp.Symbol("z")
alpha = sp.Rational(1024)
t = 2
n = 1  # This seems unused in A_j but is in the notebook logic

In [73]:
ms = []
ms.append(sp.Matrix([[1, 0], [0, 1]]))
ms.append(sp.Matrix([[1 + z, 0], [0, 1]]))
ms.append(sp.Matrix([[1, z], [0, 1]]))
ms.append(sp.Matrix([[1, 0], [z, 1 + z]]))
ms.append(sp.Matrix([[1 + z, 0], [0, 1 + z]]))
ms.append(sp.Matrix([[1, z], [z, 1]]))

In [74]:
# Define A_j generator
def A_j(P):
    z_alpha = z / alpha
    num = P.subs(z, z_alpha)
    denom = z_alpha**t + (1 - z) * num
    return num / denom

In [75]:
def R_func(M):
    return M[0, 0] + M[1, 1] - M[0, 1] - M[1, 0]

def Q_func(M):
    return M[0, 0] * M[1, 1] - M[0, 1] * M[1, 0]

def A_jk(M):
    z_alpha = z / alpha
    Q_eval = Q_func(M).subs(z, z_alpha)
    R_eval = R_func(M).subs(z, z_alpha)
    numerator = Q_eval
    denominator = (1 - z) * Q_eval + z_alpha**t * R_eval
    return numerator / denominator

# Generate the 6 generating functions
A_jks = [A_jk(M) for M in ms]

In [76]:
# Generate the 6 generating functions
A_jks = [A_jk(M) for M in ms]

def nth_coefficient_from_R(R_expr, n_val=2**21):
    z = sp.symbols('z')
    R_expr = sp.simplify(R_expr)
    
    P_expr, Q_expr = sp.fraction(R_expr)
    P = sp.poly(P_expr, z)
    Q = sp.poly(Q_expr, z)

    # Solve Q(z) = 0
    z_roots = sp.solve(Q, z)
    
    # rho_k = 1 / z_k
    rhos = [1 / root for root in z_roots]
    Q_prime = Q.diff()

    a_values = []
    for rho in rhos:
        z_inv = 1 / rho
        numerator = -rho * P.as_expr().subs(z, z_inv)
        denominator = Q_prime.as_expr().subs(z, z_inv)
        a_k = sp.simplify(numerator / denominator)
        a_values.append(a_k)

    # Final result: sum(a_k * rho_k^n)
    # We perform the sum symbolically then evalf closely
    result = sum(a * rho**n_val for a, rho in zip(a_values, rhos))
    
    # KEY IMPROVEMENT: 
    # 1. Use higher precision (100 digits)
    # 2. Explicitly take the real part sp.re() to remove imaginary noise
    return sp.re(result.evalf(35))


In [77]:
print("Calculating coefficients (this may take a moment)...")
A_jks_coefficient = [nth_coefficient_from_R(A_jk) for A_jk in A_jks]

# Ns from notebook
sigma = 2**10
n1 = sigma * (sigma - 1)**2 * (sigma - 2)
n2 = 2 * sigma * (sigma - 1) * (sigma - 2)
n3 = 2 * sigma * (sigma - 1) * (sigma - 2)
n4 = 4 * sigma * (sigma - 1)
n5 = sigma * (sigma - 1)
n6 = sigma * (sigma - 1)
Ns = [n1, n2, n3, n4, n5, n6]

weighted_score = [coeff * n for coeff, n in zip(A_jks_coefficient, Ns)]

Calculating coefficients (this may take a moment)...


In [78]:
import time
t0 = time.time()
P_distinct = sp.Poly(1, z)
P_repeat = sp.Poly(1 + z, z)
Aj_distinct = A_j(P_distinct.as_expr())
Aj_repeat = A_j(P_repeat.as_expr())
E_x = (alpha**2 - alpha) * distinct_coeff + alpha * repeat_coeff
t1 = time.time()


repeat_coeff = nth_coefficient_from_R(Aj_repeat)
distinct_coeff = nth_coefficient_from_R(Aj_distinct)


Var_x = sum(weighted_score) + E_x - E_x**2
t2 = time.time()

print("\nResults:")
print(f"Expectation (E_x): {E_x}")
print(f"Variance (Var_x):  {Var_x}")
print(f"Std Dev:           {sp.re(Var_x**0.5)}")

print(f"Time for Expectation: {t1 - t0:.4f} seconds")
print(f"Time for Variance:    {t2 - t1:.4f} seconds (cumulative after E_x)")


Results:
Expectation (E_x): 141909.32995500691890740108161117408
Variance (Var_x):  84368.326461630490855798936347282507
Std Dev:           290.46226340375179768668162964091382
Time for Expectation: 0.0014 seconds
Time for Variance:    0.0832 seconds (cumulative after E_x)


In [ ]:
# 290.462 263 403 751 797 69
# 290.462 263 403 751 797 68 668 1629 640 913 82